In [38]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

df = pd.read_excel('/users/amulya/Desktop/TAM.xlsx')
unwanted_columns = ['Website', 'Company Linkedin Url', 'Lists', '# Employees',
       'Industry', 'Account Owner', 'Facebook Url', 'Twitter Url',
       'Company City', 'Company State', 'Company Country',
       'Company Postal Code', 'Company Address', 'Keywords', 'Company Phone',
       'SEO Description', 'Technologies', 'Total Funding', 'Latest Funding',
       'Latest Funding Amount', 'Last Raised At', 'Annual Revenue',
       'Number of Retail Locations', 'Apollo Account Id', 'SIC Codes',
       'Short Description', 'Founded Year']
df = df.drop(unwanted_columns, axis = 1)
# print(df.columns)
df.fillna(0, inplace=True)
null_values = df.isnull().sum()
# print(null_values)
unique_values = df['Company'].unique()
# a = df['Company'].value_counts()
# print(unique_values)
company_dummies = pd.get_dummies(df['Company'])
df = df.drop('Company',axis = 1)
df = pd.concat([df, company_dummies], axis=1)
ICP_dummies = pd.get_dummies(df['ICP'])
df = df.drop('ICP',axis = 1)
df = pd.concat([df, ICP_dummies], axis=1)
ICP_dummies = pd.get_dummies(df['Outsourcing From \n(Eastern European Countries)'])
df = df.drop('Outsourcing From \n(Eastern European Countries)',axis = 1)
df = pd.concat([df, ICP_dummies], axis=1)
design_dummies = pd.get_dummies(df['Design Head'],prefix = 'Design_Head')
df = df.drop('Design Head',axis = 1)
df = pd.concat([df, design_dummies], axis=1)
linkedin_dummies = pd.get_dummies(df['Activities over Linkedin ( 30-60-90; >90 DAYS)'], prefix = 'linkedin')
df = df.drop('Activities over Linkedin ( 30-60-90; >90 DAYS)',axis = 1)
df = pd.concat([df, linkedin_dummies], axis=1)
growth_dummies = pd.get_dummies(df['HC Growth \n(1 yr)'], prefix = 'growth')
df = df.drop('HC Growth \n(1 yr)',axis = 1)
df = pd.concat([df, growth_dummies], axis=1)
df.columns = df.columns.astype(str)
param_grid = {
    'n_clusters': [3, 4, 5, 6, 7,8,9,10,15],
    'max_iter': [100, 200, 300]
}
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
data = kmeans.fit(df)
cluster_labels = kmeans.labels_
centroids = kmeans.cluster_centers_
a = kmeans.n_iter_
print(a)
df['Cluster'] = cluster_labels
silhouette_score = silhouette_score(df, cluster_labels)
print("Silhouette Score:", silhouette_score)
print(df['Cluster'].value_counts())

3
Silhouette Score: 0.41855016291878916
Cluster
1    128
4     77
0     58
3     25
2      9
Name: count, dtype: int64


1. selected 5 columns
2. replaced all the null values with 0
3. converting comapny column to numerical column

In [54]:
param_grid = {
    'n_clusters': [3, 4, 5, 6, 7],  # Values of K to try
    'init': ['k-means++', 'random'],  # Initialization methods
    'max_iter': [100, 200, 300]  # Maximum number of iterations
}
kmeans = KMeans(random_state=42)
# Create GridSearchCV instance
grid_search = GridSearchCV(kmeans,param_grid, cv=5, scoring='silhouette')

# Fit the grid search to your data
grid_search.fit(df)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Get the best silhouette score
best_score = grid_search.best_score_
print("Best Silhouette Score:", best_score)

# Get the best estimator
best_estimator = grid_search.best_estimator_

# Use the best estimator to fit the data
best_estimator.fit(df)

InvalidParameterError: The 'scoring' parameter of GridSearchCV must be a str among {'f1_weighted', 'jaccard', 'neg_mean_squared_log_error', 'neg_mean_gamma_deviance', 'neg_brier_score', 'f1', 'f1_micro', 'explained_variance', 'max_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'jaccard_samples', 'neg_median_absolute_error', 'balanced_accuracy', 'fowlkes_mallows_score', 'roc_auc_ovo', 'rand_score', 'homogeneity_score', 'neg_root_mean_squared_log_error', 'neg_mean_absolute_error', 'precision_samples', 'neg_mean_absolute_percentage_error', 'jaccard_micro', 'jaccard_weighted', 'roc_auc_ovr_weighted', 'neg_log_loss', 'top_k_accuracy', 'recall', 'matthews_corrcoef', 'roc_auc_ovr', 'roc_auc', 'accuracy', 'f1_macro', 'neg_mean_squared_error', 'f1_samples', 'adjusted_mutual_info_score', 'roc_auc_ovo_weighted', 'v_measure_score', 'neg_mean_poisson_deviance', 'normalized_mutual_info_score', 'precision_macro', 'average_precision', 'positive_likelihood_ratio', 'r2', 'adjust